# TODO
#### Klassen protecten
```
import AutoMLWrapper.automlwrapper as aw
aw.Configuration.Configuration
```

#### Erweiterung um Bibliotheken
-  @__library.setter
    - a) Global/GlobalConfig.yaml
        - lib-names
        - type mappings
    - b) directory durchsuchen 
- import in AutoMLWrapper anpassen
- SEDAR automl.py config locations anpassen

In [1]:
import pandas as pd
#from automlwrapper import AutoMLWrapper
from AutoMLWrapper.automlwrapper import AutoMLWrapper


/home/max/miniconda3/envs/AutoML/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-23 13:41:58.867326: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-23 13:41:59.443491: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-23 13:41:59.443564: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugi

Using TensorFlow backend


In [2]:
df = pd.read_csv('data/glass.csv')
user_hp = {'num_trials': 10, 'time_limit': 111, 'testtest': 123}


In [3]:
ask = AutoMLWrapper('autosklearn')
ask.SetOutputDirectory('test/ask')

if 0:
    ask.Train(data=df, target_column='Type',
        task_type='classification',
        data_type='tabular',
        problem_type='multiclass',
        hyperparameters=user_hp
    )

In [4]:
agl = AutoMLWrapper('autogluon')
agl.SetOutputDirectory('test/agl')
if 0:
    agl.Train(data=df, target_column='Type',
       task_type='classification',
       data_type='tabular',
       problem_type='multiclass',
       hyperparameters=user_hp
   )


In [5]:
akr = AutoMLWrapper('autokeras')
akr.SetOutputDirectory('test/akr')
if 1:
    akr.Train(data=df, target_column='Type',
        task_type='classification',
        data_type='tabular',
        problem_type='multiclass',
        hyperparameters=user_hp
    )

Trial 10 Complete [00h 00m 05s]
val_accuracy: 0.9545454382896423

Best val_accuracy So Far: 0.9545454382896423
Total elapsed time: 00h 00m 56s
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/54
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
7/7 [==============================] - 0s 1ms/step - loss: 1.8360 - accuracy: 0.0561
Epoch 2/54
7/7 [==============================] - 0s 813us/step - loss: 1.7641 - accuracy: 0.1215
Epoch 3/54
7/7 [==============================] - 0s 962us/step - loss: 1.7130 - accuracy: 0.2336
Epoch 4/54
7/7 [==============================] - 0s 846us/step - loss: 1.6684 - accuracy: 0.3131
Epoch 5/54
7/7 [==============================] - 0s 817us/step - loss: 1.6289 - accuracy: 0.3551
Epoch 6/54
7/7 [==============================] - 0s 895us/step - loss: 1.5934 - accuracy: 0.4299
Epoch 7/54
7/7 [=======================

...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: smac attempted to use a functionality that requires module emcee, but it couldn't be loaded. Please install emcee and retry.
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


INFO:tensorflow:Assets written to: test/akr/structured_data_classifier/best_model/assets


INFO:tensorflow:Assets written to: test/akr/structured_data_classifier/best_model/assets
